In [1]:
import dlib

In [2]:
import cv2
import glob
import random
import math
import numpy as np
import itertools
from sklearn.svm import SVC

In [3]:
# Initialize the Emotions set
!wget https://github.com/AKSHAYUBHAT/TensorFace/raw/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat

emotions = ["anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Emotion list
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") #Or set this to whatever you named the downloaded file
clf = SVC(kernel='linear', probability=True, tol=1e-3)#, verbose = True) #Set the classifier as a support vector machines with polynomial kernel
data = {} #Make dictionary for all values
#data['landmarks_vectorised'] = []

--2020-09-24 07:18:11--  https://github.com/AKSHAYUBHAT/TensorFace/raw/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AKSHAYUBHAT/TensorFace/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat [following]
--2020-09-24 07:18:11--  https://raw.githubusercontent.com/AKSHAYUBHAT/TensorFace/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99693937 (95M) [application/octet-stream]
Saving to: ‘shape_predictor_68_face_landmarks.dat’

shape_predictor_68_ 100%[==

In [4]:
!wget https://github.com/WuJie1010/Facial-Expression-Recognition.Pytorch/archive/master.zip
!unzip master.zip
!rm -rf master.zip

--2020-09-24 07:18:15--  https://github.com/WuJie1010/Facial-Expression-Recognition.Pytorch/archive/master.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/WuJie1010/Facial-Expression-Recognition.Pytorch/zip/master [following]
--2020-09-24 07:18:15--  https://codeload.github.com/WuJie1010/Facial-Expression-Recognition.Pytorch/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [  <=>               ]   5.58M  18.3MB/s    in 0.3s    

2020-09-24 07:18:15 (18.3 MB/s) - ‘master.zip’ saved [5854288]

Archive:  master.zip
fefe6653f3e09912693a73c54e6a8247adca3090
   creating: Faci

In [5]:
def get_files(emotion): #Define function to get file list, randomly shuffle it and split 80/20
    files = glob.glob("/content/Facial-Expression-Recognition.Pytorch-master/CK+48/%s/*" %emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] #get first 80% of file list
    prediction = files[-int(len(files)*0.2):] #get last 20% of file list
    return training, prediction
def get_landmarks(image):
    detections = detector(image, 1)
    for k,d in enumerate(detections): #For all detected face instances individually
        shape = predictor(image, d) #Draw Facial Landmarks with the predictor class       
        xlist = []
        ylist = []
        for i in range(1,68): #Store X and Y coordinates in two lists
          xlist.append(float(shape.part(i).x))
          ylist.append(float(shape.part(i).y))
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]
        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(w)
            landmarks_vectorised.append(z)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            landmarks_vectorised.append(dist)
            landmarks_vectorised.append((math.atan2(y, x)*360)/(2*math.pi))
        data['landmarks_vectorised'] = landmarks_vectorised
    if len(detections) < 1:
        data['landmarks_vestorised'] = "error"
def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for emotion in emotions:
        print(" working on %s" %emotion)
        training, prediction = get_files(emotion)
        #Append data to training and prediction list, and generate labels 0-7
        for item in training:
            image = cv2.imread(item) #open image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
            clahe_image = clahe.apply(gray)
            get_landmarks(clahe_image)
            if data['landmarks_vectorised'] == "error":
                print("no face detected on this one")
            else:
                training_data.append(data['landmarks_vectorised']) #append image array to training data list
                training_labels.append(emotions.index(emotion))
        for item in prediction:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            clahe_image = clahe.apply(gray)
            get_landmarks(clahe_image)
            if data['landmarks_vectorised'] == "error":
                print("no face detected on this one")
            else:
                prediction_data.append(data['landmarks_vectorised'])
                prediction_labels.append(emotions.index(emotion))
               
    return training_data, training_labels, prediction_data, prediction_labels
accur_lin = []
for i in range(0,10):
    print("Making sets %s" %i) #Make sets by random sampling 80/20%
    training_data, training_labels, prediction_data, prediction_labels = make_sets()
    npar_train = np.array(training_data) #Turn the training set into a numpy array for the classifier
    npar_trainlabs = np.array(training_labels)
    print("training SVM linear %s" %i) #train SVM
    clf.fit(npar_train, training_labels)
    print("getting accuracies %s" %i) #Use score() function to get accuracy
    npar_pred = np.array(prediction_data)
    pred_lin = clf.score(npar_pred, prediction_labels)
    print ("linear: ", pred_lin)
    accur_lin.append(pred_lin) #Store accuracy in a list
print("Mean value lin svm: %s" %np.mean(accur_lin)) #FGet mean accuracy of the 10 runs

Making sets 0
 working on anger
 working on contempt
 working on disgust
 working on fear
 working on happy
 working on sadness
 working on surprise
training SVM linear 0
getting accuracies 0
linear:  0.8601036269430051
Making sets 1
 working on anger
 working on contempt
 working on disgust
 working on fear
 working on happy
 working on sadness
 working on surprise
training SVM linear 1
getting accuracies 1
linear:  0.8704663212435233
Making sets 2
 working on anger
 working on contempt
 working on disgust
 working on fear
 working on happy
 working on sadness
 working on surprise
training SVM linear 2
getting accuracies 2
linear:  0.8756476683937824
Making sets 3
 working on anger
 working on contempt
 working on disgust
 working on fear
 working on happy
 working on sadness
 working on surprise
training SVM linear 3
getting accuracies 3
linear:  0.8756476683937824
Making sets 4
 working on anger
 working on contempt
 working on disgust
 working on fear
 working on happy
 working on 

In [6]:
# Get the prediction value of specific object

output_pred = clf.predict_proba(training_data)[102]


# Displaying the Emotions normalized values for specfied emotions
print(output_pred[0])
print(output_pred[1])
print(output_pred[2])
print(output_pred[3])
print(output_pred[4])
print(output_pred[5])
print(output_pred[6])

0.6962729922889324
0.02780848320246583
0.025584211841248525
0.03361530003586177
0.005378100233228206
0.18444214416869775
0.026898768229565655


In [7]:
# Access the mmaximum score
obtained_score = np.amax(output_pred)
position= np.where(output_pred == np.amax(output_pred))[0][0]
position

0

In [8]:
emotions[position]
print('Captured Emotion  -->  '+emotions[position]+' has obtained :: '+str(obtained_score*100)+'% ')

Captured Emotion  -->  anger has obtained :: 69.62729922889324% 


In [9]:
# HMM Development

states = ('Depressed', 'Non-Depressed')

observations = ( 'positive-emotion', 'negative-emotion')

start_probabilities = {
    'Depressed' : {'Non-Depressed':0.2, 'Depressed':0.8},
    'Non-Depressed':{'Non-Depressed':0.5,'Depressed':0.5 }
}

emission_probabilites = {
    'Depressed':{'positive-emotion':0.3,'negative-emotion':0.7},
    'Non-Depressed':{'positive-emotion':0.6,'negative-emotion':0.4}   
}

In [10]:
!pip install hmmlearn
from hmmlearn import hmm

     |████████████████████████████████| 368kB 4.4MB/s 


In [11]:
modelHMM = hmm.MultinomialHMM(n_components=2)

users = [{'userid':1000, 'observation':1},{'userid':1001, 'observation':0}]


# Define Start Probability Vecctor
modelHMM.startprob_ = np.array([0.2, 0.8])

# Define Transmat Vector
modelHMM.transmat_ = np.array([[0.2, 0.8],
                            [0.5, 0.5]])

# Define emission Vector
modelHMM.emissionprob_ = np.array([[0.3, 0.7],[0.6, 0.4]])

# Model Fit
#modelHMM.fit([[]])

# Previous State [Non-Depression 1, Depression 0 ]
observation=np.array([[1]]).T

modelHMM.predict_proba([[1]])

#TODO: Save the response to Firebase
#TODO: Retreive observations by Firebase

array([[0.30434783, 0.69565217]])